In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

## Store Part 1 results into DataFrame 

In [3]:
csv_path = os.path.join("output_data","cities.csv")

df = pd.read_csv(csv_path)

df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Zunil,14.7833,-91.4833,53.49,93,8,8.68,GT,1642923745
1,1,Albany,42.6001,-73.9662,20.73,67,100,3.94,US,1642923745
2,2,Mahajanga,-15.7167,46.3167,80.62,89,40,14.97,MG,1642923745
3,3,Guerrero Negro,27.9769,-114.0611,59.47,73,2,4.29,MX,1642923746
4,4,Vaini,-21.2000,-175.2000,80.76,78,20,9.22,TO,1642923746


## Humidity Heatmap

In [37]:
gmaps.configure(api_key=g_key)

coords = df[['Lat', 'Lng']]
humidity = df['Humidity']

    
layout = {
    'width': '700px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto',
}

fig = gmaps.figure(layout=layout)

heat_layer = gmaps.heatmap_layer(coords, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

## Weather Criteria

In [40]:
wind = 10
temp = [70, 80]
cloud = 0

ideal_df = df[(df['Wind Speed'] < 10) & (df['Max Temp'] < temp[1]) & (df['Max Temp'] > temp[0]) & (df["Cloudiness"] == cloud)]

ideal_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
281,281,Kīhei,20.7850,-156.4656,75.85,80,0,9.22,US,1642923854
364,364,Black River,18.0264,-77.8487,72.99,86,0,5.03,JM,1642923887
375,375,Kahului,20.8947,-156.4700,75.85,80,0,9.22,US,1642923891
410,410,Lagos,6.5833,3.7500,73.72,63,0,3.83,NG,1642923687
411,411,Bull Savanna,17.8868,-77.5902,70.45,83,0,4.59,JM,1642923906
481,481,Pochutla,15.7432,-96.4661,70.83,53,0,4.36,MX,1642923936
555,555,Puerto Escondido,15.8500,-97.0667,74.59,67,0,5.70,MX,1642923966


## Hotel Map

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure